# Linear models and Neural Networks
In this notebook, we will learn what is the difference between a linear model and a neural network. We will use both models for predictions on an articial dataset and for hand written digits classification.


#### Imports and functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math 
import seaborn as sns
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_blobs, make_regression
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.base import BaseEstimator
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn import metrics
from sklearn.pipeline import make_pipeline



In [ ]:
def display_confusion_matrix(y_test, y_pred):
  confusion_matrix = pd.DataFrame(metrics.confusion_matrix(y_test, y_pred))
  confusion_matrix.index.name = 'Actual'
  confusion_matrix.columns.name = 'Predicted'
  sns.heatmap(confusion_matrix, annot=True)

def plot_decision_boundary(X, y, clf: BaseEstimator):
  xx, yy = np.mgrid[0:1:.01, 0:1:.001]
  grid = np.c_[xx.ravel(), yy.ravel()]
  f, ax = plt.subplots(figsize=(8, 6))
  probs = clf.predict_proba(grid)[:, 1].reshape(xx.shape)
  contour = ax.contourf(xx, yy, probs, 25, cmap="RdBu",
                        vmin=0, vmax=1)
  ax_c = f.colorbar(contour)
  ax_c.set_label("$P(y = 1)$")
  ax_c.set_ticks([0, .25, .5, .75, 1])

  ax.scatter(X[100:,0], X[100:, 1], c=y[100:], s=50,
            cmap="RdBu", vmin=-.2, vmax=1.2,
            edgecolor="white", linewidth=1)

  ax.set(aspect="equal",
        xlim=(0, 1), ylim=(0, 1),
        xlabel="$X_1$", ylabel="$X_2$")

## Regression

First, we will learn how a linear regression model works. To understand this, let us first generate some articial data of two dimensions - `x` is the feature and `y` is the predicted value. Our goal will be to build a model which correctly predicts the value of `y` given `x`.

In [ ]:
x, y = make_regression(n_samples=1000, n_features=1, noise=30, bias=2, random_state=0)
x = x[:, 0]
sns.scatterplot(x=x, y=y)

### Manual selection of linear coefficients

Now, let us try to manually find the best line which fits this dataset.
To do this, we will try to "guess" the coefficients of a linear function which defines the predictions.

In our case, we have only 1 feature `x`, so we will try to find coefficients $a$ and $b$ for equation $y=ax+b$.

When selecting the coefficients, we will try to minimize the mean squarred prediction error.

Try to find the best coefficients by starting with $a=0, b=0$. Is it a good fit?

In [ ]:
a = ??
b = ??
y_pred = x * a + b

mse = mean_squared_error(y, y_pred)
sns.scatterplot(x=x, y=y)
sns.lineplot(x=x, y=y_pred, color='r').set_title(f"$y={a}x+{b}$, MSE: {mse:.2f}")

Now, try to gradually modify the values of `a` and `b`. Can you find the best fit which minimizes the error?

We can plot the error for different values of parameter $a$. The optimal values of parameters could be found by finding the coefficients that minimize this error (loss) function. However, when there are many variables, such a direct optimization would be inefficient and approximate gradient optimization methods are applied instead.

In [ ]:
b = 0
a_grid = np.linspace(-100, 200)
mse_for_a_grid = []

for a in a_grid:
  y_pred = x * a + b
  mse = mean_squared_error(y, y_pred)
  mse_for_a_grid.append(mse)
sns.lineplot(x=a_grid, y=mse_for_a_grid)

### Linear regression model

When choosing the best parameters, you started with some random guess, next, you looked at the results and you updated the parameters a little bit until the classification error decreased. 

A simple linear model which is often used for regression tasks is the *linear regression* model. The learning works in a similar way - it tries to choose the parameters of a linear function: $$\hat{y}=a_1x_1 + a_2x_2 \ldots +a_nx_n +b$$
by minimizing the squarred error between the true and predicted value of $y$. After each iteration, it calculates the errors and updated weights according to the gradients of the objective function. This algorithm is called *gradient descent*.

Let us check how the model fits to the same dataset.

In [ ]:
x_reshaped = x.reshape(-1, 1)
lr = LinearRegression(fit_intercept=True)

Fit the model and calculate predictions.

In [ ]:
lr.fit(x_reshaped, y)
y_pred = lr.predict(x_reshaped)

In [ ]:
mse = mean_squared_error(y, y_pred)
sns.scatterplot(x=x, y=y)
sns.lineplot(x=x, y=y_pred, color='r').set_title(
    f"$y={lr.coef_[0]:.2f}x+{lr.intercept_:.2f}$, MSE: {mse:.2f}")

## Classification

Next, we will compare the performance of linear and non-linear models on the classification tasks.

### What is a linearly separable problem?

To understand the difference between linear and non-linear models, let us generate some artificial data points for a binary classification problem.

Our goal will be to separate the two classes (0 and 1).

In [ ]:
X, y = make_blobs(n_samples=1000, n_features=2, centers=2, cluster_std=3,
                  random_state=1)
X = minmax_scale(X)
sns.scatterplot(x=X[:,0], y=X[:,1], hue=y)

### Linear classification with logistic regression

A linear model which is often used for classification is the *logistic regression* model. It applies a logistic (sigmoid) function to the output of linear function to calculate the probablity of each class for a given point:

$$ \sigma(z)= \frac{1}{1-e^{-z}}$$

Next, the predictions are classified as positive if this score is above a theshold (typically 0.5)
 



In [ ]:
x = np.linspace(-10, 10, 100) 
z = 1/(1 + np.exp(-x)) 
  
sns.lineplot(x, z) 
plt.xlabel("z") 
plt.ylabel("$\sigma(z)$") 

Let us apply the logistic classifier to this binary classification problem.

Split the dataset into train and test sets and calculate the classification accuracy.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
lr = LogisticRegression()
lr.fit(??)
y_pred = lr.predict(??)

Next, we will plot the decision boundary of the model (its prediction score for every point on the grid).

In [ ]:
plot_decision_boundary(X, y, lr)

In [ ]:
display_confusion_matrix(y_test, y_pred)

Since the model fits linear coefficients, it is possible to display the resulting weights to verify which ones have the highest impact on prediction (positive or negative).

In [ ]:
pd.Series(lr.coef_[0], index=['x1', 'x2'])

### Non-linearly separable problem

In the previous example, we could find the line which separates the examples. Now, let's look at a different dataset:

In [ ]:
X, y = make_moons(n_samples=5000, noise=0.3, random_state=0)
X = minmax_scale(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)
sns.scatterplot(x=X[:,0], y=X[:,1], hue=y)

Could you plot a line which separates these two classes? 

Now, let's fit a logistic regression model on the train set and calculate the accuracy on the test set.

In [ ]:
lr = ??
??
y_pred = ??

Plot the decision boundary for this model:

In [ ]:
plot_decision_boundary(X_test, y_test, lr)

In [ ]:
display_confusion_matrix(y_test, y_pred)

We can see that in this case, the accuracy is much lower and the line does not separate all the points. 

This is because this dataset is not **linearly separable**. To find a proper decision boundary, we will need a non-linear model which can learn more complex features, such as a neural network. 

## Neural network classification

To see the difference between a linear classifier and a neural network, let us take a look at the following examples of model training for different architectures and datasets. 

In each example, look at:
- Look at the train and test loss after (approx.) 10, 100 and 1000 epochs.
- Modify the learning rate parameter (check the values of 0.01, 0.1 and 1). How did the loss change after these epochs?



[Example 1](https://playground.tensorflow.org/#activation=sigmoid&regularization=L2&batchSize=10&dataset=gauss&regDataset=reg-plane&learningRate=0.01&regularizationRate=0.001&noise=15&networkShape=&seed=0.32144&showTestData=true&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false): linearly separable problem, linear model and features

[Example 2](https://playground.tensorflow.org/#activation=sigmoid&regularization=L2&batchSize=16&dataset=xor&regDataset=reg-plane&learningRate=0.01&regularizationRate=0.001&noise=15&networkShape=&seed=0.98211&showTestData=true&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false): non-linearly separable problem, linear model and features

[Example 3](https://playground.tensorflow.org/#activation=sigmoid&regularization=L2&batchSize=16&dataset=xor&regDataset=reg-plane&learningRate=0.01&regularizationRate=0.001&noise=15&networkShape=&seed=0.98211&showTestData=true&discretize=false&percTrainData=50&x=true&y=true&xTimesY=true&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false): non-linearly separable problem, linear model, non-linear features

[Example 4](https://playground.tensorflow.org/#activation=relu&regularization=L2&batchSize=16&dataset=xor&regDataset=reg-plane&learningRate=0.01&regularizationRate=0.001&noise=10&networkShape=4,3,1&seed=0.87204&showTestData=true&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false): non-linearly separable problem, non-linear model, linear features.

Analyze what features were learnt by each neuron of the multi-layer model. What can you observe about the features in each layer?

Try to experiment with other datasets and input features. 

### Neural network implementation

Next, we will implement the neural network architecture which learns the non-linear features for the previous dataset. We will use `MLPClassifier` class from sklearn https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

In [ ]:
nn =  MLPClassifier(solver='lbfgs', alpha=1, random_state=1, max_iter=5000,
            hidden_layer_sizes=[20, 10],  learning_rate_init=0.01, 
            early_stopping=True)
?? # fit the model on training set
?? # print the accuracy on the test set
?? # plot the decision boundary

Try to modify the network architecture by changing the number of neurons and layers in the model (`hidden_layer_sizes`). For example, add one more layer and change the number of neurons in layers to 100, 20, 10. How has the decision boundary changed?

Calculate the predictions, accuracy and display the confusion matrix.

In [ ]:
??

In [ ]:
??

## Comparing the classifiers

Now, apply the `LogisticRegression` and `MLPClassifier` models to the same dataset that we used with `DecisionTreeClassifier`. Which model gives better results (accuracy)?

In [ ]:
from sklearn.datasets import load_breast_cancer

breast_cancer_dataset = load_breast_cancer()
breast_cancer_dataset_pd = pd.DataFrame(breast_cancer_dataset.data, columns=breast_cancer_dataset.feature_names)
X = breast_cancer_dataset_pd
y = breast_cancer_dataset.target

breast_cancer_dataset_pd.shape

(569, 30)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
# split the X and y matrices into train and test sets
lr = LogisticRegression(max_iter=5000) 
?? # train LR model on the train set
nn = MLPClassifier()
?? # train MLP model on the train set
?? # print the accuracy on the test set for both models

Object `` not found.
Object `` not found.


Display the weights for the features in the LR model (sorted by their positive impact)

In [ ]:
pd.Series(lr.coef_[0], index=breast_cancer_dataset.feature_names).sort_values(ascending=False)

## Hand-written digits classification

Next, we will compare the linear and non-linear models on the task of hand-writtem digit image classification. We will use a popular MNIST dataset from:
https://www.openml.org/d/554

Let's plot examples of images for each each digit from this dataset.

In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

for i in range(10):
  plt.imshow(X[y==str(i)][0].reshape(28, 28))
  plt.show()


First, we will reshape the digits (pixels from each image) into a flat numerical vector - these will be the features for the model.

We will split the dataset into train and test splits (using a sample of 10000 examples for faster computations). We will use a standard scaler to scale the features.

In [ ]:
X = X.reshape((X.shape[0], -1))
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=10000,
                                                    test_size=2000)
scaler = StandardScaler()
lr = LogisticRegression(penalty='l1', solver='saga', tol=0.1, C=0.1)
image_classification_pipeline = make_pipeline(scaler, lr)

In [ ]:
?? # fit the pipeline on the training set
?? # predict classes for the test set
?? # print accuracy for the test set
?? # display the confusion matrix 

### Classifier weights analysis

One advantage of a linear classifier is its interpretability - we can plot the weights of the features to learn which ones have the strongest positive (blue) and negative (red) impact on the prediction for each class.

In [ ]:
coef = lr.coef_.copy()
scale = np.abs(coef).max()
for i in range(10):
  plt.imshow(coef[i].reshape(28, 28), 
           interpolation='nearest', cmap=plt.cm.RdBu, vmin=-scale, vmax=scale)
  plt.show()

Now, we will train a non-linear neural network model (multi-layer perceptron) and compare the results with the linear model.

In [ ]:
nn =  MLPClassifier(solver='lbfgs', alpha=0.1, random_state=1, max_iter=2000,
            early_stopping=True, hidden_layer_sizes=[10, 5])

?? # create the pipeline with scaler and MultiPerceptron classifier
?? # fit the pipeline
?? # predict classes for the test set
?? # print accuracy
?? # display the confusion matrix

The neural network should give a higher accuracy, but it is more difficult to interpret its weights (though there are some methods that enable this, eg. LIME which we used on the previous lab).



---

---





# What to remember
* A linear model works well for *linearly separable*  classification problems.
* A logistic regression model is a simple linear classifier which may be applied for such tasks. It learns weights for features that may be easily interpreted.
* Deep neural network models consist of multiple units (neurons) that may be organized into layers. They can learn more complex (non-linear) features and achieve higher accuracy on complex tasks. However, it is more difficult to interpret their weights.